In [ ]:
from __future__ import absolute_import, division, print_function, unicode_literals
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap, LinearSegmentedColormap
%matplotlib inline
import numpy as np
import pandas as pd
import os, shutil, glob, sys, math, cv2, re

import segmentation_models_pytorch as smp
import albumentations as albu
from torchsummary import summary

from tqdm import tqdm

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.utils.checkpoint as cp
from collections import OrderedDict
from torch.utils.model_zoo import load_url as load_state_dict_from_url
from torch import Tensor
from torch.jit.annotations import List
from torchvision import models

from torch.utils.data import DataLoader
from torch.utils.data import Dataset as BaseDataset

from sklearn.model_selection import train_test_split

In [ ]:
DATA_FOLDER = "/home/Tsung/pathology/data/tcga"

# model

In [ ]:
model = models.resnet18()
model.fc = nn.Linear(in_features=512, out_features=2, bias=True)
model.load_state_dict(torch.load("2020_09_22_18_52_46_Resnet18-tumor-or-nonTumor_Dataset_Zenodo.h5"))
model.cuda()
model.eval()

# dataset

In [ ]:
def get_augmentation():
    """Add paddings to make image shape divisible by 32"""
    test_transform = [
        albu.Resize(height = 256, width = 256, always_apply=True),
    ]
    return albu.Compose(test_transform)


def to_tensor(x, **kwargs):
    return x.transpose(2, 0, 1).astype('float32')

# https://github.com/pytorch/vision/blob/master/torchvision/transforms/functional.py, to_tensor     
def to0_1(x, **kwargs):
    return x/255

def get_preprocessing():

    _transform = [
        albu.Lambda(image=to_tensor, mask=to_tensor),
        albu.Lambda(image=to0_1, mask=to0_1),
    ]
    return albu.Compose(_transform)

In [ ]:
from PIL import Image
class Dataset(BaseDataset):
    
    def __init__(self, image_array, augmentation=None, preprocessing=None):
        self.image_array = image_array
        self.augmentation = augmentation
        self.preprocessing = preprocessing
    
    def __getitem__(self, i):
        fp = self.image_array[i]
        
        image = Image.open(fp)
        image = np.asarray(image)
        # apply augmentations
        if self.augmentation:
            sample = self.augmentation(image=image)
            image = sample['image']
        
        # apply preprocessing
        if self.preprocessing:
            sample = self.preprocessing(image=image)
            image = sample['image']
        
        return fp, image
    
    def __len__(self):
        return len(self.image_array)

In [ ]:
patches_folder = os.path.join(DATA_FOLDER, 'normalize_patches')
all_folders = sorted(os.listdir(patches_folder))

In [ ]:
for number, folder_name in enumerate(all_folders):
    if folder_name[13] == '1':
        continue
    all_images = sorted(glob.glob(os.path.join(patches_folder, folder_name, "*.jpg")))
    
    test_dataset = Dataset(
        all_images,
        augmentation = get_augmentation(),
        preprocessing = get_preprocessing()
    )
    test_dataloader = DataLoader(test_dataset, batch_size=32, shuffle=False, num_workers=0)
    
    is_tumor_image_name = np.array([])
    
    for data in test_dataloader:
        image_names, images = data
        image_names = np.array([name for name in image_names])
        
        images = images.cuda()
        
        with torch.no_grad():
            pred = model(images)
        pred = torch.softmax(pred, axis = 1)
        pred = torch.argmax(pred, axis = 1)
        pred = pred.detach().cpu().numpy()
        
        if len(pred.shape) != 1:
            pred = pred.squeeze()
            
        tp = np.where(pred == 1)[0]
        is_tumor_image_name = np.concatenate((is_tumor_image_name, image_names[tp]))
    print("{}, all_images: {}, tumor_images: {}".format(number, len(all_images), len(is_tumor_image_name)))
    np.save("{}/normalized_tumor/{}.npy".format(DATA_FOLDER, folder_name), is_tumor_image_name)

In [ ]:
npy = glob.glob("/nfs/Shared/data/tcga/tumor/*.npy")
c = 0
for i in npy:
    j = np.load(i)
    c += len(j)

# for visualize

In [ ]:
import openslide
from PIL import Image

In [ ]:
svs_folder = '/home/Tsung/pathology/data/tcga/svs/'
patches_folder = '/home/Tsung/pathology/data/without_msimss/224dense/patches/'
mask_folder = '/home/Tsung/pathology/data/without_msimss/224dense/masks/'

In [ ]:
# 數量500多，有些沒有images
import pandas as pd
df = pd.read_csv('./data/coad_Mutation_Count.txt', delimiter="\t")
cohort_count_dict = {row[1]: row[3] for index,row in df.iterrows()}

In [ ]:
import openslide
from PIL import Image
import glob
import os 
import numpy as np
svs_folder = '/home/Tsung/pathology/data/tcga/svs/'
patches_folder = '/home/Tsung/pathology/data/without_msimss/224dense/patches/'
mask_folder = '/home/Tsung/pathology/data/without_msimss/224dense/masks/'

# for i in glob.glob(os.path.join(svs_folder, '20px' , "*.svs")):
#     name = i.split("/")[-1]
#     slide = openslide.open_slide(i)
#     tile = np.array(slide.read_region((0,0), 0, slide.dimensions))
#     tile = Image.fromarray(tile)
#     tile = tile.convert("RGB")
#     tile.save(os.path.join("/home/Tsung/pathology/data/tcga/svs/20px_image", name+".jpg"))
#     print(os.path.join("/home/Tsung/pathology/data/tcga/svs/20px_image", name+".jpg"))
    
for idx, i in enumerate(glob.glob(os.path.join(svs_folder, '40px' , "*.svs"))):
    name = i.split("/")[-1]
    if os.path.exists(os.path.join("/home/Tsung/pathology/data/tcga/svs/40px_image", name+".jpg")):
        continue
    slide = openslide.open_slide(i)
    tile = np.array(slide.read_region((0,0), 0, slide.dimensions))
    tile = Image.fromarray(tile)
    tile = tile.convert("RGB")
    width, height = tile.size
    width = int(width/2)
    height = int(height/2)
    tile = tile.resize((width, height))  
    tile.save(os.path.join("/home/Tsung/pathology/data/tcga/svs/40px_image", name+".jpg"))
    print(idx, os.path.join("/home/Tsung/pathology/data/tcga/svs/40px_image", name+".jpg"))
    

In [ ]:
glob.glob("/home/Tsung/pathology/data/tcga/svs")